<a href="https://colab.research.google.com/github/FelippeSantuzzi/1-Per-odo-de-Intelig-ncia-Artificial---Fatec-Rio-Claro-SP/blob/main/Chat_bot_CAF.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q --upgrade langchain langchain-google-genai google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00


Aula 01

Importacao da API_KEY

In [2]:
from google.colab import userdata
from langchain_google_genai import ChatGoogleGenerativeAI
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')


Conexao com o google gemini

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY,

)

In [4]:
resp_text = llm.invoke("Quem é voce?")
print(resp_text.content)



Eu sou um modelo de linguagem grande, treinado pelo Google.


In [5]:
TRIAGEM_PROMPT = (
    "Você é um vendedor externo para política de vendas externas da Caf Máquinas. "
    "Dada a mensagem do usuário, retorne SOMENTE um JSON com:\n"
    "{\n"
    '  "decisao": "AUTO_RESOLVER" | "PEDIR_INFO" | "ABRIR_CHAMADO",\n'
    '  "urgencia": "BAIXA" | "MEDIA" | "ALTA",\n'
    '  "campos_faltantes": ["..."]\n'
    "}\n"
    "Regras:\n"
    '- **AUTO_RESOLVER**: Perguntas claras sobre questionamento ou procedimentos descritos nas políticas de vendas (Ex: "Posso utilizar essa maquina para moer quantos quilos de carne", "Como funciona o parcelamento de vendas?").\n'
    '- **PEDIR_INFO**: Mensagens vagas ou que faltam informações para identificar o tema ou contexto (Ex: "Preciso de ajuda como comprar uma máquina", "Tenho uma dúvida geral").\n'
    '- **ABRIR_CHAMADO**: Pedidos de exceção, liberação, aprovação ou acesso especial, ou quando o usuário explicitamente pede para abrir um chamado (Ex: "Quero exceção de modo como a máquina trabalha com a moagem de outro tipo de material.", "Solicito liberação para anexos externos.", "Por favor, abra um chamado para o setor de vendas".").'
    "Analise a mensagem e decida a ação mais apropriada."
)

In [6]:
from pydantic import BaseModel, Field
from typing import Literal, List, Dict

class triagemOut(BaseModel):
  decisao: Literal["AUTO_RESOLVER","PEDIR_INFO","ABRIR_CHAMADO"]
  urgencia: Literal["BAIXA","MEDIA","ALTA"]
  campos_faltantes: List[str] = Field(default_factory=list)

In [7]:
llm_triagem = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0,
    api_key=GOOGLE_API_KEY,

)

In [8]:
from langchain_core.messages import SystemMessage, HumanMessage

triagem_chain = llm_triagem.with_structured_output(triagemOut)

def triagem(mensagem: str) -> Dict:
  saida: TriagemOut = triagem_chain.invoke([
      SystemMessage(content=TRIAGEM_PROMPT),
      HumanMessage(content=mensagem)
  ])

  return saida.model_dump()

In [9]:
testes = ["posso utilizar essa máquina para moer quantos quilos de carne?" ,
          "preciso de ajuda como comprar essa máquina",
          " tenho uma ajuda geral",
          "por que preciso dessa máquina?"]


In [10]:
for msg_test in testes:
  print(msg_test)
  print(triagem(msg_test))
  print(f"pergunta {msg_test}\n -> Resposta: {triagem(msg_test)}\n")

posso utilizar essa máquina para moer quantos quilos de carne?
{'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}
pergunta posso utilizar essa máquina para moer quantos quilos de carne?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

preciso de ajuda como comprar essa máquina
{'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}
pergunta preciso de ajuda como comprar essa máquina
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}

 tenho uma ajuda geral
{'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}
pergunta  tenho uma ajuda geral
 -> Resposta: {'decisao': 'PEDIR_INFO', 'urgencia': 'BAIXA', 'campos_faltantes': []}

por que preciso dessa máquina?
{'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}
pergunta por que preciso dessa máquina?
 -> Resposta: {'decisao': 'AUTO_RESOLVER', 'urgencia': 'BAIXA', 'campos_faltantes': []}

